In [3]:

# Setup the dependencies
using Pkg;Pkg.add(url="https://github.com/JuDO-dev/AirBorne.jl#dev");
Pkg.add(["Dates","Plots","DataFrames"])
# using Dates: DateTime,datetime2unix
using AirBorne, Plots, Dates
using AirBorne.ETL.YFinance: get_interday_data
using AirBorne.Engines.DEDS: run
using AirBorne.Markets.StaticMarket: execute_orders!, expose_data
using AirBorne.Strategies.SMA: interday_initialize!, interday_trading_logic!
using AirBorne.Structures: summarizePerformance,TimeEvent
# using Plots: plot, plot!
unix(x) = string(round(Int, datetime2unix(DateTime(x))))
data = get_interday_data(["AAPL","GOOG"], unix("2017-01-01"), unix("2022-01-01"))
sma_initialize!(context) = interday_initialize!(context; longHorizon=20, shortHorizon=5)
sma_trading_logic! = interday_trading_logic!
evaluationEvents = [TimeEvent(t, "data_transfer") for t in sort(unique(data.date); rev=true)]
context = run(data, sma_initialize!, sma_trading_logic!, execute_orders!, expose_data;
    audit=true,max_date=DateTime(2022, 1, 1), verbose=true, initialEvents=evaluationEvents)
results = summarizePerformance(data, context)
println("Done")
plot(results.date, results.dollarValue, title="Portfolio Value - Template SMA", 
    label="value", linewidth=3, xlabel="date",ylabel="Portfolio Value")

    Updating git-repo `https://github.com/JuDO-dev/AirBorne.jl#dev`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Done-12-30T10:30:00


LoadError: syntax: invalid interpolation syntax: "$ "

In [6]:

plotd = plot(results.date, results.dollarValue ./results.dollarValue[1], title="Portfolio Value - Template SMA", 
    label="value", linewidth=3, xlabel="date",ylabel="Portfolio Value")
fig_dir = joinpath(@__DIR__, "figures")
savefig(plotd,joinpath(fig_dir,"E1_02_all_comparison.svg"))

LoadError: UndefVarError: fig_dir not defined